### Fantasy Football Player Performance Analysis
#### Objective: Compare actual player scoring vs major fantasy football site predictions

#  📚 1. Import Libraries


In [3]:
import pandas as pd
import numpy as np
import requests
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import matplotlib.pyplot as plt
import seaborn as sns

In [27]:
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

True

# 🔄 2. Data Collection

In [ ]:
# Example function for collecting data from APIs (replace with actual API calls)
def get_fantasy_data(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error fetching data from {api_url}")
        return None

# Replace with actual API URLs for ESPN, FantasyPros, CBS, Sleeper, Yahoo
espn_data = get_fantasy_data("https://api.espn.com/fantasy-football-data")
fantasypros_data = get_fantasy_data("https://api.fantasypros.com/fantasy-football-data")

# Additional APIs for other platforms (CBS, Sleeper, Yahoo)

# Convert API data to DataFrames (placeholder for now, replace with actual data structures)
espn_df = pd.DataFrame(espn_data)
fantasypros_df = pd.DataFrame(fantasypros_data)

# ESPN Fantasy Football API

In [21]:
from espn_api.football import League

swid = os.getenv("SWID")
espn_s2 = os.getenv("ESPN_S2")
league_id = os.getenv("LEAGUE_ID")

# Replace with your league ID and ESPN_S2 and SWID cookies
league = League(league_id=league_id, year=2024, espn_s2=espn_s2, swid=swid)

# Get the player pool
player_pool = league.player_map

# Create a list to hold player projections and actual scores
player_scores = []

# Fetch projected and actual scores for each player in the pool
for player in player_pool:
    print(player)
    # Get player's projected points (if available)
    projected_points = player.projected_points if hasattr(player, 'projected_points') else 0
    actual_points = player.points if hasattr(player, 'points') else 0  # Actual points may vary based on availability

    # Append player data to the list
    player_scores.append({
        'Player Name': player.name,
        'Team': player.team,
        'Position': player.position,
        'Projected Points': projected_points,
        'Actual Points': actual_points
    })

# Convert to DataFrame for easier manipulation (optional)
import pandas as pd

player_scores_df = pd.DataFrame(player_scores)

# Display the scores
print(player_scores_df)

# Optionally, save to a CSV file
player_scores_df.to_csv('player_scores.csv', index=False)


4242896


AttributeError: 'int' object has no attribute 'name'

In [ ]:
from espn_api.football import League

swid = os.getenv("SWID")
espn_s2 = os.getenv("ESPN_S2")
league_id = os.getenv("LEAGUE_ID")

# Replace with your league ID and ESPN_S2 and SWID cookies
league = League(league_id=league_id, year=2024, espn_s2=espn_s2, swid=swid)

# Fetch all the teams in the league
teams = league.teams

# Example of getting weekly projected and actual scoring
for team in teams:
    print(f"Team: {team.team_name}")
    
    # Get weekly scoring data (projected and actual)
    for matchup in team.matchups:
        print(f"Week: {matchup.week}")
        print(f"Projected Points: {matchup.projected_score}")
        print(f"Actual Points: {matchup.score}")
        print("-" * 20)  # Separator for readability


In [43]:
team_abbrev = {
    0:'New York Wise Guys',
    1:'S-Rank Showstopper',
    2:"Dennis's Daring Team",
    3:'Just OK',
    4:'Huntsvegas',
    5:'Boofaloof',
    6:'Beer For My Horses',
    7:'HRts Tua Good',
    8:'Cam',
    9:'Fantasy Football Team '
}

# make sure these map to your roster settings
# we have a two qb league but QB2 is an OP incase you can roster 2 QBs in a week
eligible_positions = {
    0: 'QB',
    1: 'TQB',
    2: 'RB',
    3: 'RB/WR',
    4: 'WR',
    5: 'WR/TE',
    6: 'TE',
    7: 'OP',
    8: 'DT',
    9: 'DE',
    10: 'LB',
    11: 'DL',
    12: 'CB',
    13: 'S',
    14: 'DB',
    15: 'DP',
    16: 'D/ST',
    17: 'K',
    18: 'P',
    19: 'HC',
    20: 'BE',
    21: 'IR',
    22: '',
    23: 'RB/WR/TE',
    24: 'ER',
    25: 'Rookie',
    'QB': 0,
    'RB': 2,
    'WR': 4,
    'TE': 6,
    'D/ST': 16,
    'K': 17,
    'FLEX': 23,
    'DT': 8,
    'DE': 9,
    'LB': 10,
    'DL': 11,
    'CB': 12,
    'S': 13,
    'DB': 14,
    'DP': 15,
    'HC': 19
}

PRO_TEAM_MAP = {
    0 : 'None',
    1 : 'ATL',
    2 : 'BUF',
    3 : 'CHI',
    4 : 'CIN',
    5 : 'CLE',
    6 : 'DAL',
    7 : 'DEN',
    8 : 'DET',
    9 : 'GB',
    10: 'TEN',
    11: 'IND',
    12: 'KC',
    13: 'LV',
    14: 'LAR',
    15: 'MIA',
    16: 'MIN',
    17: 'NE',
    18: 'NO',
    19: 'NYG',
    20: 'NYJ',
    21: 'PHI',
    22: 'ARI',
    23: 'PIT',
    24: 'LAC',
    25: 'SF',
    26: 'SEA',
    27: 'TB',
    28: 'WSH',
    29: 'CAR',
    30: 'JAX',
    33: 'BAL',
    34: 'HOU'
}

In [48]:
season=2024
url = 'https://lm-api-reads.fantasy.espn.com/apis/v3/games/ffl/seasons/{}/segments/0/leagues/{}?view=mBoxscore&view=mMatchupScore&view=mRoster&view=mSettings&view=mStatus&view=mTeam&view=modular&view=mNav'.format(season, league_id)
# create an empty list to append data to
projection_data = []
for week in range(1, 6): #change this to 18 if you want to include the playoffs
    print('Week',week)
 
    # get the raw data
    r = requests.get(url,
                     params={'scoringPeriodId': week},
                     cookies={'SWID': swid, 'espn_s2': espn_s2})
    espn_raw_data = r.json()
 
    # loop over each team in the request
    for team in espn_raw_data['teams']:
        # get the team_id so we can map it to team_names
        team_id = team['id']
        # create a new column with the team_abbrev using the team_id as the key
        team_owner = team_abbrev[team_id - 1]
        # loop over every player on the teams roster
        for player in team['roster']['entries']:
            espn_id = player['playerPoolEntry']['player']['id']
            player_name = player['playerPoolEntry']['player']['fullName']
            nfl_team = PRO_TEAM_MAP[player['playerPoolEntry']['player']['proTeamId']]
            lineup_slot = player['lineupSlotId']
            # create a new column with the position using the lineup_slot as the key
            position = eligible_positions[lineup_slot]
            
            # get the projected and actual points
            projected = None, 
            actual = None
            # loop over the stats for each player
            for stats in player['playerPoolEntry']['player']['stats']:
                # skip the rows where the scoring period does not match up with the curren week
                if stats['scoringPeriodId'] != week:
                    continue
                # if the source id = 0 then these are actual stats
                if stats['statSourceId'] == 0:
                    actual = stats['appliedTotal']
                # if the source id = 1 then these are projected stats
                elif stats['statSourceId'] == 1:
                    projected = stats['appliedTotal']
            
            print(player_name)
            print(actual)
            print(projected)
            # append all the data to the empty list
            projection_data.append([
                week, 
                team_owner,
                espn_id, 
                player_name, 
                nfl_team,
                lineup_slot, 
                position, 
                projected, 
                actual
            ])
 
print('Done getting projected & actual stats.')

Week 1
Amon-Ra St. Brown
4.3
19.409320863100003
Kyren Williams
14.4
15.687108454
Patrick Mahomes
15.14
19.466517012
George Kittle
8.0
11.349451505000001
Najee Harris
8.9
11.332226628000003
Stefon Diggs
21.9
14.283708341099999
Devin Singletary
9.2
11.959614969
Aaron Rodgers
8.58
16.165505542899997
Jake Elliott
10.0
8.425354911000001
Jayden Reed
33.1
11.8122120249
Jets D/ST
-1.0
3.8546357419999993
Adam Thielen
7.9
11.179327556000002
Jerome Ford
18.9
11.890800856000002
Antonio Gibson
1.8
8.476058363
Taysom Hill
4.6
7.20862204656
Blake Corum
0.0
5.689339944199999
CeeDee Lamb
13.600000000000001
20.2953622469
Jahmyr Gibbs
17.4
16.589690961000002
Marvin Harrison Jr.
1.4
15.403083057
Josh Jacobs
12.4
15.768539555
Kenneth Walker III
18.9
16.110538766
Cooper Kupp
32.0
14.7684978
Kyler Murray
14.18
17.890927111
Terry McLaurin
3.7
13.215460625
David Montgomery
16.299999999999997
12.711824387000004
Raheem Mostert
3.9
11.228485960000002
Brock Bowers
11.8
9.771022105
Jayden Daniels
28.16
17.821076242

In [49]:
# convert the list to a dataframe with the following column names
df = pd.DataFrame(projection_data, 
   columns=['Week', 
   'Team',
   'ESPN ID', 
   'Player', 
   'NFL Team',
   'Lineup Slot', 
   'Position', 
   'Projected', 
   'Actual'])
# sample ten random rows of data
df.sample(10)

,Week,Team,ESPN ID,Player,NFL Team,Lineup Slot,Position,Projected,Actual
761,5,Beer For My Horses,4040655,Darnell Mooney,ATL,23,RB/WR/TE,12.430122,31.5
445,3,HRts Tua Good,3932905,Diontae Johnson,CAR,20,BE,11.740700,26.2
696,5,Dennis's Daring Team,3915486,Tyler Conklin,NYJ,6,TE,7.999696,11.5
413,3,Boofaloof,4360248,Kyle Pitts,ATL,6,TE,9.569561,7.9
71,1,Huntsvegas,4429615,Zay Flowers,BAL,23,RB/WR/TE,14.836059,11.1
214,2,Just OK,3054850,Alvin Kamara,NO,2,RB,16.509264,44.0
325,3,New York Wise Guys,4430737,Kyren Williams,LAR,2,RB,18.760423,31.6
697,5,Dennis's Daring Team,3128390,Allen Lazard,NYJ,20,BE,7.772504,13.4
255,2,Boofaloof,-16033,Ravens D/ST,BAL,16,D/ST,7.545059,9.0
493,4,New York Wise Guys,4040761,Devin Singletary,NYG,2,RB,14.326028,4.8


In [47]:
player['playerPoolEntry']['player']

{'active': True,
 'defaultPositionId': 2,
 'draftRanksByRankType': {'STANDARD': {'auctionValue': 38,
   'published': False,
   'rank': 17,
   'rankSourceId': 0,
   'rankType': 'STANDARD',
   'slotId': 0},
  'PPR': {'auctionValue': 0,
   'published': False,
   'rank': 2468,
   'rankSourceId': 0,
   'rankType': 'PPR',
   'slotId': 0}},
 'droppable': True,
 'eligibleSlots': [2, 3, 23, 7, 20, 21],
 'firstName': 'Isiah',
 'fullName': 'Isiah Pacheco',
 'id': 4361529,
 'injured': True,
 'injuryStatus': 'INJURY_RESERVE',
 'lastName': 'Pacheco',
 'lastNewsDate': 1726688829000,
 'lastVideoDate': 1726583477000,
 'outlooks': {'outlooksByWeek': {'1': "Isiah Pacheco took on a more significant role in the Kansas City offense in his second NFL season, averaging 14.6 carries over 14 regular-season games (up from 10.0 a game as a rookie) and just missing his first career 1,000-yard rushing campaign. The Rutgers product's physical running style did lead to a couple missed games due to injuries, but he ha

# NFL Data Python Package

In [13]:
import nfl_data_py as nfl

In [14]:
player_id_list = nfl.import_ids()

In [19]:
filtered_row = player_id_list.loc[player_id_list['sleeper_id'] == 2279]

# Fantasy Pros

In [11]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from io import StringIO

def fetch_fantasypros_fpts_with_ids(positions=["QB"], weeks=[1], year=2024):
    """
    Fetches FantasyPros weekly fantasy points (FPTS) predictions for multiple positions and weeks,
    and extracts player IDs from the class attribute (e.g., fp-id-22902).
    
    Parameters:
    - positions (list): List of positions to fetch ('QB', 'RB', 'WR', 'TE', 'K', 'DST')
    - weeks (list): List of weeks for which to fetch predictions.
    - year (int): The NFL season year (e.g., 2024).
    
    Returns:
    - pd.DataFrame: Combined DataFrame containing player names, IDs, and FPTS for all positions and weeks.
    - list: List of DataFrames, one for each position and week.
    """
    
    all_data = []  # List to store DataFrames for each week and position
    
    for position in positions:
        for week in weeks:
            # Adjust URL based on position: use PPR scoring for RB, WR, TE
            if position.upper() in ["RB", "WR", "TE"]:
                url = f"https://www.fantasypros.com/nfl/projections/{position}.php?scoring=PPR&week={week}&year={year}"
            else:
                url = f"https://www.fantasypros.com/nfl/projections/{position}.php?week={week}&year={year}"
            
            # Send request and get HTML content
            try:
                response = requests.get(url)
                response.raise_for_status()
            except requests.exceptions.HTTPError as err:
                print(f"HTTP error occurred for {position}, week {week}: {err}")
                continue
            except Exception as err:
                print(f"Error occurred for {position}, week {week}: {err}")
                continue
            
            # Parse the HTML using BeautifulSoup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Convert the parsed HTML into a string and wrap it in a StringIO object
            html_string = str(soup)
            html_io = StringIO(html_string)  # Wrap in StringIO to avoid the warning
            
            # Extract the player table using pandas
            df = pd.read_html(html_io)[0]
            
            # Flatten multi-level column names, if any, by joining them with underscores
            df.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col for col in df.columns]
            
            # Find the "FPTS" column under "MISC"
            fpts_column = [col for col in df.columns if "MISC_FPTS" in col]
            if not fpts_column:
                print(f"FPTS column not found for {position}, week {week}")
                continue
            
            # Rename the player column to something easier to work with and filter for FPTS
            df.rename(columns={"Unnamed: 0_level_0_Player": "Player"}, inplace=True)
            
            # Select only Player and the FPTS column
            df = df[['Player'] + fpts_column]
            
            # Now process the player information with BeautifulSoup
            player_data = []
            player_table = soup.find('table')  # Assuming the player table is the first one
            
            for row in player_table.find_all('tr')[1:]:  # Skip the header row
                player_cell = row.find('a', class_="fp-player-link")  # Find the anchor with class "fp-player-link"
                
                if player_cell:
                    player_name = player_cell.text.strip()
                    player_class = player_cell.get('class', [])  # Get class attribute
                    
                    # Extract the player_id from the class (e.g., 'fp-id-17233')
                    player_id = None
                    for class_name in player_class:
                        if 'fp-id-' in class_name:
                            player_id = class_name.split('-')[-1]  # Extract the numeric part after 'fp-id-'
                            break
                    
                    # Add the player data to the list, ensuring player_id is captured
                    player_data.append({'player_name': player_name, 'player_id': player_id})
            
            # Convert player_data into a DataFrame
            player_df = pd.DataFrame(player_data)

            # Add a "Week" and "Position" column to identify which week and position the data is for
            df['Week'] = week
            df['Position'] = position
            player_df['Week'] = week
            player_df['Position'] = position

            # Reset the index to ensure uniqueness before concatenation
            df = df.reset_index(drop=True)
            player_df = player_df.reset_index(drop=True)

            # Ensure both DataFrames have the same number of rows before concatenating
            if len(df) == len(player_df):
                # Merge the DataFrames by index
                merged_df = pd.concat([df, player_df], axis=1)
                # Append the merged DataFrame for this week and position to the list
                all_data.append(merged_df)
            else:
                print(f"Data mismatch for {position}, week {week}. Skipping.")
    
    # Combine all weekly DataFrames for all positions into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    
    return combined_df, all_data

# Example usage:
positions_to_fetch = ["qb","rb", "wr", "te"]  # Fetch data for RBs, WRs, and TEs
weeks_to_fetch = [1, 2, 3, 4, 5, 6]  # Fetch data for weeks 1, 2, and 3
projections_all_positions, all_dfs = fetch_fantasypros_fpts_with_ids(positions=positions_to_fetch, weeks=weeks_to_fetch, year=2024)

# Display the combined DataFrame for all positions and weeks
print(projections_all_positions.head())


                  Player  MISC_FPTS  Week Position         player_name  \
0         Josh Allen BUF       24.3     1       qb          Josh Allen   
1        Jalen Hurts PHI       22.2     1       qb         Jalen Hurts   
2  Patrick Mahomes II KC       20.4     1       qb  Patrick Mahomes II   
3      Lamar Jackson BAL       20.3     1       qb       Lamar Jackson   
4        C.J. Stroud HOU       19.8     1       qb         C.J. Stroud   

  player_id  Week Position  
0     17298     1       qb  
1     19275     1       qb  
2     16413     1       qb  
3     17233     1       qb  
4     23071     1       qb  


# Sleeper API

In [20]:
from sleeper_wrapper import Stats
stats = Stats()
week_stats = stats.get_week_projections("regular",2018, 5)
score = stats.get_player_week_score(week_stats, "DET")

In [48]:
import pprint
season_type="regular" 
position=None
order_by="pts_ppr"
# Base URL for Sleeper projections API
base_url = f"https://api.sleeper.com/projections/nfl/{season}"

# Add week parameter if provided
if week is not None:
    base_url += f"/{week}"

# Construct the query parameters
params = {
    "season_type": season_type,
    "order_by": order_by
}

# Add position filter if provided
if position:
    params["position"] = position

# Fetch the projections data from Sleeper
response = requests.get(base_url, params=params)

if response.status_code != 200:
    print(f"Failed to fetch data from Sleeper API. Status code: {response.status_code}")
    #return None

# Parse the JSON response
projections_data = response.json()
pprint.pprint(projections_data[0])

{'category': 'proj',
 'company': 'rotowire',
 'date': '2023-09-10',
 'game_id': '202310121',
 'last_modified': 1694490322648,
 'opponent': 'NE',
 'player': {'fantasy_positions': ['QB'],
            'first_name': 'Jalen',
            'injury_body_part': None,
            'injury_notes': None,
            'injury_start_date': None,
            'injury_status': None,
            'last_name': 'Hurts',
            'metadata': {'channel_id': '1113708715887849472',
                         'rookie_year': '2020'},
            'news_updated': 1728884111747,
            'position': 'QB',
            'team': 'PHI',
            'team_abbr': None,
            'team_changed_at': None,
            'years_exp': 4},
 'player_id': '6904',
 'season': '2023',
 'season_type': 'regular',
 'sport': 'nfl',
 'stats': {'adp_dd_ppr': 16.0,
           'cmp_pct': 65.21,
           'fum': 0.41,
           'fum_lost': 0.18,
           'gp': 1.0,
           'pass_2pt': 0.06,
           'pass_att': 27.5,
           'p

In [54]:
import requests
import pandas as pd

def get_sleeper_projections(season, weeks, positions, season_type='regular'):
    # List to store data for all weeks and positions
    all_data = []
    
    # Loop over the weeks
    for week in weeks:
        # Loop over the positions
        for position in positions:
            # Format the URL for the Sleeper projections API
            url = f"https://api.sleeper.com/projections/nfl/{season}/{week}?season_type={season_type}&position={position}"
            
            # Send a request to the Sleeper API
            response = requests.get(url)
            
            # Parse the JSON response
            data = response.json()
            
            # Extract relevant information
            players_data = []
            for projection in data:
                player_info = projection['player']
                player_stats = projection['stats']
                
                # Ensure we have the needed data
                if 'pts_ppr' in player_stats:
                    players_data.append({
                        'player_id': projection['player_id'],
                        'first_name': player_info['first_name'],
                        'last_name': player_info['last_name'],
                        'team': player_info['team'],
                        'position': player_info['position'],
                        'fantasy_points_ppr': player_stats['pts_ppr'],
                        'week': week,  # Add week information
                        'position': position  # Add position information
                    })
            
            # Create a DataFrame for the current position and week
            position_df = pd.DataFrame(players_data)
            
            # Add to the list of all weeks' and positions' data
            all_data.append(position_df)
    
    # Combine all data into a single DataFrame
    combined_df = pd.concat(all_data, ignore_index=True)
    
    # Add a full player name column
    combined_df['player_name'] = combined_df['first_name'] + ' ' + combined_df['last_name']
    
    # Set player_id as index
    combined_df.set_index('player_id', inplace=True)
    
    # Select relevant columns
    combined_df = combined_df[['player_name', 'team', 'position', 'fantasy_points_ppr', 'week']]
    
    return combined_df

# Example usage
season = '2023'
weeks = [1, 2, 3]  # Multiple weeks
positions = ['QB', 'RB', 'WR', 'TE']  # Multiple positions
df = get_sleeper_projections(season, weeks, positions)
print(df.head())

               player_name  team position  fantasy_points_ppr  week
player_id                                                          
1049        Ryan Tannehill  None       QB               15.94     1
1166          Kirk Cousins   ATL       QB               20.16     1
1234        Russell Wilson   PIT       QB               18.84     1
1373            Geno Smith   SEA       QB               19.26     1
1837       Jimmy Garoppolo   LAR       QB               17.14     1
